In [12]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget
if 'NOTEBOOK_INITIATED_FLAG' not in globals():
    NOTEBOOK_INITIATED_FLAG = True
    %cd ..

import os 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image
import matplotlib.colors as colors

import examplesetup
import lib.deconvolution as deconvolution
import lib.utils as utils
import lib.xtrace as xtrace
import lib.datagen as datagen
import cv2
from numpy.random import default_rng
rng = default_rng()

import tensorflow as tf
from tensorflow import keras
import numpy as np

model_name = "superres_nodeconv_abs"
model = keras.models.load_model(f'trained_networks/{model_name}.h5')

#img = image.imread("example_image.png").sum(axis=2)
img_9 = np.load('example_files/test_data_9.npy')/1e6
img_0 = np.load('example_files/test_data_0.npy')/1e6

img, true_img = datagen.SuperRes(1, 1, (400, 400), 4)[0]
img = img[0,:,:,0]
true_img = true_img[0,:,:,0]
recovered_img = model(img[np.newaxis,:,:, np.newaxis], training=False).numpy()[0,:,:,0]
recovered_img_0 = model(img_0[np.newaxis,:,:, np.newaxis], training=False).numpy()[0,:,:,0]
recovered_img_9 = model(img_9[np.newaxis,:,:, np.newaxis], training=False).numpy()[0,:,:,0]
percent = 99
perc = np.percentile(img_0, percent)
#G = xtrace.depth_spill_psf(datagen.global_config, *utils.ray_grid(datagen.global_config["dimensions"]))
#recovered_img_conv_0 = deconvolution.richard_lucy(distorted_img_0, G, 6)

fig, axs = plt.subplots(1, 3, figsize=(10, 5), sharex=True, sharey=True)
axs[0].imshow(true_img, vmin=0, vmax=perc)
axs[0].set_title("True Image")
axs[1].imshow(img, vmin=0, vmax=perc,extent=[0.5, 400, 400, 0.5])
axs[1].set_title("Small Img")
axs[2].imshow(recovered_img, vmin=0, vmax=perc)
axs[2].set_title("Recovered")

fig, axs = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)
axs[0,0].imshow(img_0, vmin=0, vmax=perc, extent=[0.5, 400, 400, 0.5])
axs[0,0].set_title("Detector")
axs[0,1].imshow(recovered_img_0, vmin=0, vmax=perc)
axs[0,1].set_title("Recovered")
axs[1,0].imshow(img_9, vmin=0, vmax=perc, extent=[0.5, 400, 400, 0.5])
axs[1,0].set_title("Detector")
axs[1,1].imshow(recovered_img_9, vmin=0, vmax=perc)
axs[1,1].set_title("Recovered")

G = xtrace.depth_spill_psf(datagen.global_config, *utils.ray_grid(datagen.global_config["dimensions"]))
recovered_img_conv_0 = deconvolution.richard_lucy(img_0, G, 6)
distorted_conv = (G.get()@recovered_img_conv_0.flatten()).reshape(recovered_img_conv_0.shape)

config, _ = utils.upsample_transform(datagen.global_config, np.zeros(1), 4)
G_up = xtrace.depth_spill_psf(config, *utils.ray_grid(config["dimensions"]))
distorted_ml = utils.downsample_img((G_up.get()@recovered_img_0.flatten()).reshape(recovered_img_0.shape), 4)

mm = np.max(img_0)
diff_rl = np.abs(img_0 - distorted_conv)/mm
#diff_rl = np.sign(diff_rl)*np.sqrt(np.abs(diff_rl))
diff_ml = np.abs(img_0 - distorted_ml)/mm
#diff_ml = np.sign(diff_ml)*np.sqrt(np.abs(diff_ml))


vmax = np.max(np.maximum(diff_rl, diff_ml))
norm = colors.PowerNorm(gamma=0.5, vmin=0, vmax=vmax)
fig, axs = plt.subplots(1, 2, figsize=(20, 10), sharex=True, sharey=True, constrained_layout=True)
#pcm_1 = axs[0].imshow(distorted_img_0, vmax=perc, vmin=0)
pcm = axs[0].imshow(diff_rl, norm=norm)
axs[0].set_title("Difference (original - RL)")
axs[1].imshow(diff_ml, norm=norm)
axs[1].set_title("Difference (original - ML)")
fig.colorbar(pcm, ax=axs, shrink=1, location='bottom')
plt.show()

plt.figure()
plt.imshow(img_0)
plt.title("Original Image")
plt.colorbar()


#Add local energy loss constraint? (ex: 10x10 area should contain about the same ammount of energy)
#Add positivity loss constraint?
#Try the Pecent contraint with lower randomness value -> trains on more realistic schenarios

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …